# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 12:57PM,253004,19,ADV80009002,"AdvaGen Pharma, Ltd",Released
1,Dec 13 2022 12:57PM,253004,19,ADV80009003,"AdvaGen Pharma, Ltd",Released
2,Dec 13 2022 12:57PM,253004,19,ADV80009004,"AdvaGen Pharma, Ltd",Released
3,Dec 13 2022 12:57PM,253004,19,ADV80009005,"AdvaGen Pharma, Ltd",Released
4,Dec 13 2022 12:57PM,253004,19,ADV80009007,"AdvaGen Pharma, Ltd",Released
5,Dec 13 2022 12:57PM,253004,19,ADV80009008,"AdvaGen Pharma, Ltd",Released
6,Dec 13 2022 12:57PM,253004,19,ADV80009009,"AdvaGen Pharma, Ltd",Released
7,Dec 13 2022 12:57PM,253004,19,ADV80009010,"AdvaGen Pharma, Ltd",Released
8,Dec 13 2022 12:57PM,253004,19,ADV80009011,"AdvaGen Pharma, Ltd",Released
9,Dec 13 2022 12:57PM,253004,19,ADV80009012,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,253001,Released,9
46,253002,Released,13
47,253003,Released,10
48,253004,Released,12
49,253005,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253001,NaN,NaN,9.0
253002,NaN,NaN,13.0
253003,NaN,NaN,10.0
253004,NaN,NaN,12.0
253005,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0.0,0.0,1.0
252861,1.0,3.0,1.0
252891,1.0,0.0,4.0
252899,0.0,0.0,1.0
252927,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0,0,1
252861,1,3,1
252891,1,0,4
252899,0,0,1
252927,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,0,0,1
1,252861,1,3,1
2,252891,1,0,4
3,252899,0,0,1
4,252927,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,,,1
1,252861,1,3,1
2,252891,1,,4
3,252899,,,1
4,252927,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd"
12,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC"
13,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC
23,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC
36,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC
45,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC
53,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations"
54,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC"
56,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC"
59,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd",,,12
1,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC",,,1
2,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC,,,10
3,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,,,13
4,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,,,9
5,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,,,8
6,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",,,1
7,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",,,2
8,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC",,,3
9,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd",12,,
1,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC",1,,
2,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC,10,,
3,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13,,
4,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9,,
5,Dec 13 2022 12:36PM,253000,10,MedStone Pharma LLC,8,,
6,Dec 13 2022 12:32PM,252999,20,"HVL, LLC dba Atrium Innovations",1,,
7,Dec 13 2022 12:26PM,252997,19,"Emersa Waterbox, LLC",2,,
8,Dec 13 2022 12:25PM,252996,10,"Nextsource Biotechnology, LLC",3,,
9,Dec 13 2022 12:19PM,252992,10,ISDIN Corporation,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd",12,,
1,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC",1,,
2,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC,10,,
3,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13,,
4,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd",12.0,NaN,NaN
1,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC,10.0,NaN,NaN
3,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13.0,NaN,NaN
4,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:57PM,253004,19,"AdvaGen Pharma, Ltd",12.0,0.0,0.0
1,Dec 13 2022 12:57PM,253005,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Dec 13 2022 12:48PM,253003,10,MedStone Pharma LLC,10.0,0.0,0.0
3,Dec 13 2022 12:42PM,253002,10,MedStone Pharma LLC,13.0,0.0,0.0
4,Dec 13 2022 12:39PM,253001,10,MedStone Pharma LLC,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5059382,122.0,7.0,2.0
12,505931,4.0,0.0,0.0
15,758799,75.0,41.0,47.0
16,1011821,4.0,0.0,0.0
17,252964,1.0,0.0,0.0
19,758849,15.0,0.0,0.0
20,758946,3.0,0.0,0.0
21,505971,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5059382,122.0,7.0,2.0
1,12,505931,4.0,0.0,0.0
2,15,758799,75.0,41.0,47.0
3,16,1011821,4.0,0.0,0.0
4,17,252964,1.0,0.0,0.0
5,19,758849,15.0,0.0,0.0
6,20,758946,3.0,0.0,0.0
7,21,505971,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,122.0,7.0,2.0
1,12,4.0,0.0,0.0
2,15,75.0,41.0,47.0
3,16,4.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,15.0,0.0,0.0
6,20,3.0,0.0,0.0
7,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,122.0
1,12,Released,4.0
2,15,Released,75.0
3,16,Released,4.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21
Status,,,,,,,,
Completed,2.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,41.0,0.0,0.0,0.0,0.0,1.0
Released,122.0,4.0,75.0,4.0,1.0,15.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21
0,Completed,2.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,41.0,0.0,0.0,0.0,0.0,1.0
2,Released,122.0,4.0,75.0,4.0,1.0,15.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21
0,Completed,2.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,41.0,0.0,0.0,0.0,0.0,1.0
2,Released,122.0,4.0,75.0,4.0,1.0,15.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()